In [1]:
## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


In [2]:
tss_annotation = doc['tss_annotation']

In [3]:
from collections import defaultdict
import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [4]:
meta_samples = pd.read_csv('../raw_metaSamples.tsv',sep="\t",index_col=0)
meta_samples.head()

,Tissue,Experiment,Name,Full
CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC1,CHBMDM1hKLA2_ATAC_JHS1000_SD_AGGTTGGG_S11_L001...
CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R1_001.fastq.gz,BMDM1hKLA,ATAC,BMDM1hKLA_ATAC2,CHBMDM1hKLA1_ATAC_JHS998_SD_TGGGTTTC_S9_L001_R...
CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_001.fastq.gz,BMDM1hKLA,GRO,BMDM1hKLA_GRO1,CHBMDM1hKLA_GRO_JHS1027_SD_CAAAAG_S42_L003_R1_...
CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_001.fastq.gz,BMDM1hKLA,GROCap,BMDM1hKLA_GROCap1,CHBMDM1hKLA_GRO_JHS1034_SD_AGTCAA_S49_L003_R1_...
CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001.fastq.gz,BMDMwt,GRO,BMDMwt_GRO1,CHBMDMwt_GRO_JHS1026_SD_ATTCCT_S41_L003_R1_001...


In [5]:
peaks_tissue = pd.read_csv("Results/merged/samples.merge.peaksexpression.log10",index_col=0,sep="\t")
peaks_tissue.head()

,f04_peaks/Brain_START2_and_Brain_STARTinput1,f04_peaks/Brain_GROCap1_and_Brain_GRO1,f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_STARTinput1,f04_peaks/Kidney_START1_and_Kidney_STARTinput1,f04_peaks/Muscle_START2_and_Muscle_STARTinput1,f04_peaks/Heart_START1_and_Heart_STARTinput1,f04_peaks/Lung_GROCap1_and_Lung_GRO1,f04_peaks/Liver_START2_and_Liver_STARTinput1,f04_peaks/MiscOrgans_START2_and_MiscOrgans_STARTinput1,f04_peaks/Muscle_START1_and_Muscle_STARTinput1,...,f04_peaks/Brain_START1_and_Brain_STARTinput2,f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_STARTinput1,f04_peaks/Pancreas_START2_and_Pancreas_STARTinput1,f04_peaks/Brain_START3_and_Brain_STARTinput1,f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,f04_peaks/Brain_START1_and_Brain_STARTinput1,f04_peaks/CHO_GROCap1_and_CHO_GRO1,f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,f04_peaks/Heart_START2_and_Heart_STARTinput1,f04_peaks/Lung_START1_and_Lung_STARTinput1
0,1.217484,0.786751,0.0,1.439333,1.371068,0.523746,0.0,0.000000,1.015779,0.466868,...,1.380211,1.232996,1.045323,1.100371,0.311754,1.380211,0.614897,0.755875,0.0,1.187521
1,1.025306,1.714330,0.0,0.706718,0.943495,0.670246,0.0,0.000000,0.863323,0.831230,...,0.795880,0.835691,0.761176,0.541579,2.017033,0.795880,1.672098,1.823474,0.0,1.068186
2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.332438,0.000000,0.000000,0.000000,0.0,0.000000
3,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.468347,0.000000,0.000000,0.000000,0.0,0.000000
4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.544068,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.252853,0.000000,0.0,0.000000


In [6]:
def merge_columns(df, mapping_dict):
    '''Function that merges columns by taking the mean of them.
    Merges based on if they have the same element in the meta_samples meta_column.
    Returns:
        new_df: Dataframe but with the columns of interest merged. Also column names are now based
                on the unique meta_samples[meta_column].
    '''

    vals = mapping_dict.keys()#np.unique(meta_samples[meta_column].values)
    new_df = pd.DataFrame(index=df.index, columns=vals)
    for i in vals:
        if not mapping_dict[i] == []:#(meta_samples[meta_column] == i).any():
            new_col = (df.loc[:, mapping_dict[i]])
            new_col = new_col.mean(axis=1)
            new_df.loc[:, i] = new_col
    new_df = new_df.loc[:, ~(new_df.isnull().all())]
    return new_df


In [7]:
tissue_to_peaks = defaultdict(list)
for val in peaks_tissue.columns.values:
    count = 0
    for t in np.unique(meta_samples["Tissue"].values):
        if t in val:
            count += 1
            tissue_to_peaks[t].append(val)
            if count > 1:
                print("This sample has multiple tissues associated with it")
                print("Sample: ",val)

In [8]:
peaks_tissue_merged = merge_columns(peaks_tissue,mapping_dict=tissue_to_peaks)

In [9]:
peaks_tissue_merged.head()

,Heart,BMDMwt,Spleen,FemaleReproductive,CHO,Brain,Lung,Liver,MiscOrgans,Muscle,Kidney,Pancreas
0,0.261873,0.755875,0.584573,0.616498,0.672469,1.168983,0.395840,0.710920,1.058075,0.918968,0.821805,0.856388
1,0.335123,1.823474,0.775760,0.417845,1.616393,0.919980,0.356062,0.896105,0.906356,0.887362,1.195095,0.617696
2,0.000000,0.000000,0.000000,0.000000,0.048459,0.000000,0.000000,0.211156,0.000000,0.000000,0.110813,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.029164,0.000000,0.000000,0.187034,0.000000,0.000000,0.156116,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.103864,0.000000,0.000000,0.368390,0.000000,0.000000,0.000000,0.000000


In [10]:
peaks_tissue_merged.to_csv("Results/merged/tissues.merge.peaksexpression.log10",sep="\t")

## Do the same but for txn_df 

In [37]:
txn_df = pickle.load(open("Results/tss_annotation/txn_df_02.p","rb"))

In [38]:
## invert and create a peaks_to_tissue
peaks_to_tissue = dict()
for t in tissue_to_peaks:
    for p in tissue_to_peaks[t]:
        peaks_to_tissue[os.path.basename(p)] = t

In [39]:
txn_df["Tissues"] = ""
for ind,val in tqdm.tqdm(txn_df.iterrows()):
    if len(val["samples"]) != 0:
        tis = set()
        for t in val["samples"]:
            tis.add(peaks_to_tissue[t])
        txn_df.at[ind,"Tissues"] = ",".join(tis)

52382it [00:15, 3460.73it/s]


In [40]:
pickle.dump(txn_df,open("Results/tss_annotation/txn_df_02_tissues.p","wb"))
